In [2]:
pip install openai boto3 pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Step 2: Set up OpenAI and S3 configuration
import boto3
import pandas as pd
import openai
from io import StringIO
import urllib.parse
import os

# Replace with your OpenAI API key
api_key = "Dummy"

# S3 bucket and folder configuration
bucket_name = "kiran-yelp-project-bucket"
s3_folder = "category_files/"

# List of CSV files (categories)
csv_files = [
    "american (traditional).csv",
    "bars.csv",
    "coffee & tea.csv",
    "Food.csv",
    "gyms.csv",
    "nightlife.csv",
    "restaurants.csv",
    "sandwiches.csv"
]

# Step 3: Define a function to generate GPT response with category awareness (new API format)
def generate_reply(review_text, category):
    client = openai.OpenAI(api_key=api_key)
    prompt = f"""
You're the owner of a business in the {category} category. Write a professional and polite response to this customer review:

"{review_text}"

Response:
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content

# Step 4: Loop through all category CSV files
s3 = boto3.client("s3")
for file_name in csv_files:
    encoded_file = urllib.parse.quote(file_name)
    input_key = f"{s3_folder}{file_name}"
    output_key = f"{s3_folder}gpt_{file_name}"

    category = os.path.splitext(file_name)[0]  # Extract category name from filename

    print(f"\n📥 Processing file: {file_name}")

    response = s3.get_object(Bucket=bucket_name, Key=input_key)
    df = pd.read_csv(response["Body"])
    print(f"✅ Loaded {len(df)} reviews")

    df["gpt_response"] = df["review_text"].apply(lambda x: generate_reply(x, category))
    
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=output_key, Body=csv_buffer.getvalue())

    print(f"✅ Saved GPT responses to: {output_key}")



📥 Processing file: american (traditional).csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_american (traditional).csv

📥 Processing file: bars.csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_bars.csv

📥 Processing file: coffee & tea.csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_coffee & tea.csv

📥 Processing file: Food.csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_Food.csv

📥 Processing file: gyms.csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_gyms.csv

📥 Processing file: nightlife.csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_nightlife.csv

📥 Processing file: restaurants.csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_restaurants.csv

📥 Processing file: sandwiches.csv
✅ Loaded 100 reviews
✅ Saved GPT responses to: category_files/gpt_sandwiches.csv
